In [161]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
pip install git+https://github.com/librosa/librosa

  Cloning https://github.com/librosa/librosa to /tmp/pip-req-build-2yw1y3b2
  Running command git clone -q https://github.com/librosa/librosa /tmp/pip-req-build-2yw1y3b2


In [0]:
pip install hmmlearn

In [0]:

import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm

In [0]:
def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix

def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    return mfcc

def clustering(X, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0, verbose=0)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans  

In [0]:
if __name__ == "__main__":
    class_names = ["di", "ra", "va", "yte", "nguoi","test_di", "test_ra", "test_va", "test_yte", "test_nguoi"]
    dataset = {}
    for cname in class_names:
        print(f"Load {cname} dataset")
        dataset[cname] = get_class_data(os.path.join("/content/drive/My Drive/data", cname))

    # Get all vectors in the datasets
    all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
    print("vectors", all_vectors.shape)
    # Run K-Means algorithm to get clusters
    kmeans = clustering(all_vectors)
    print("centers", kmeans.cluster_centers_.shape)

    models = {}
    for cname in class_names:
        class_vectors = dataset[cname]
        # convert all vectors to the cluster index
        # dataset['one'] = [O^1, ... O^R]
        # O^r = (c1, c2, ... ct, ... cT)
        # O^r size T x 1
        
        dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])

        if cname == 'di':
          hmm = hmmlearn.hmm.MultinomialHMM(
            n_components=6, random_state=0, n_iter=1000, verbose=True,
            startprob_prior=np.array([0.7,0.2,0.1,0.0,0.0,0.0]),
            transmat_prior=np.array([
                [0.4,0.5,0.1,0.0,0.0,0.0,],
                [0.0,0.4,0.5,0.1,0.0,0.0,],
                [0.0,0.0,0.4,0.5,0.1,0.0,],
                [0.0,0.0,0.0,0.4,0.5,0.1,],
                [0.0,0.0,0.0,0.0,0.4,0.6,],
                [0.0,0.0,0.0,0.0,0.0,1.0,],
            ]),
          )
        elif cname == 'ra':
          hmm = hmmlearn.hmm.MultinomialHMM(
            n_components=6, random_state=0, n_iter=1000, verbose=True,
            startprob_prior=np.array([0.7,0.2,0.1,0.0,0.0,0.0]),
            transmat_prior=np.array([
                [0.3,0.6,0.1,0.0,0.0,0.0,],
                [0.0,0.3,0.6,0.1,0.0,0.0,],
                [0.0,0.0,0.3,0.6,0.1,0.0,],
                [0.0,0.0,0.0,0.3,0.6,0.1,],
                [0.0,0.0,0.0,0.0,0.3,0.7,],
                [0.0,0.0,0.0,0.0,0.0,1.0,],
            ]),
          )
        elif cname == 'va':
          hmm = hmmlearn.hmm.MultinomialHMM(
            n_components=6, random_state=0, n_iter=1000, verbose=True,
            startprob_prior=np.array([0.7,0.2,0.1,0.0,0.0,0.0]),
            transmat_prior=np.array([
                [0.7,0.2,0.1,0.0,0.0,0.0,],
                [0.0,0.7,0.2,0.1,0.0,0.0,],
                [0.0,0.0,0.7,0.2,0.1,0.0,],
                [0.0,0.0,0.0,0.7,0.2,0.1,],
                [0.0,0.0,0.0,0.0,0.7,0.3,],
                [0.0,0.0,0.0,0.0,0.0,1.0,],
            ]),
          )
        elif cname == 'yte':
          hmm = hmmlearn.hmm.MultinomialHMM(
            n_components=9, random_state=0, n_iter=1000, verbose=True,
            startprob_prior=np.array([0.7,0.2,0.1,0.0,0.0,0.0,0.0,0.0,0.0]),
            transmat_prior=np.array([
                [0.2,0.5,0.2,0.1,0.0,0.0,0.0,0.0,0.0,],
                [0.0,0.2,0.5,0.2,0.1,0.0,0.0,0.0,0.0,],
                [0.0,0.0,0.2,0.5,0.2,0.1,0.0,0.0,0.0,],
                [0.0,0.0,0.0,0.2,0.5,0.2,0.1,0.0,0.0,],
                [0.0,0.0,0.0,0.0,0.2,0.5,0.2,0.1,0.0,],
                [0.0,0.0,0.0,0.0,0.0,0.2,0.5,0.2,0.1,],
                [0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.6,0.2,],
                [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.7,],
                [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,],
            ]),
          )
        elif cname == 'nguoi':
          hmm = hmmlearn.hmm.MultinomialHMM(
            n_components=12, random_state=0, n_iter=1000, verbose=True,
            startprob_prior=np.array([0.7,0.2,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]),
            transmat_prior=np.array([
                [0.4,0.5,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,],
                [0.0,0.4,0.5,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,],
                [0.0,0.0,0.4,0.5,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,],
                [0.0,0.0,0.0,0.4,0.5,0.1,0.0,0.0,0.0,0.0,0.0,0.0,],
                [0.0,0.0,0.0,0.0,0.4,0.5,0.1,0.0,0.0,0.0,0.0,0.0,],
                [0.0,0.0,0.0,0.0,0.0,0.4,0.5,0.1,0.0,0.0,0.0,0.0,],
                [0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.5,0.1,0.0,0.0,0.0,],
                [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.5,0.1,0.0,0.0,],
                [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.5,0.1,0.0,],
                [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.5,0.1,],
                [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,],
                [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,],
            ]),
          )

        if cname[:4] != 'test':
            X = np.concatenate(dataset[cname])
            lengths = list([len(x) for x in dataset[cname]])
            print("training class", cname)
            print(X.shape, lengths, len(lengths))
            hmm.fit(X, lengths=lengths)
            models[cname] = hmm
    print("Training done")

    

In [0]:
print("Testing")
for true_cname in class_names:
  for O in dataset[true_cname]:
    if true_cname[:4] == 'test':
      score = {cname : model.score(O, [len(O)]) for cname, model in models.items() if cname[:4] != 'test' }
      print(true_cname, score)

In [0]:
print("Testing")
to_test = ['test_di', 'test_ra', 'test_va', 'test_yte', 'test_nguoi']
for true_cname in to_test:
    correct = 0
    failed = 0
    real_name = true_cname.split('_')[-1]

    for O in dataset[true_cname]:
        score = {cname : model.score(O, [len(O)]) for cname, model in models.items() if cname[:4] != 'test' }

        match = True
        for key in score:
            if score[key] > score[real_name]:
                match = False
        if match:
            correct += 1
        else:
            failed += 1
            # print(real_name, score)

    acc = correct/(correct+failed)
    print(real_name + " : " + str(acc))